In [ ]:
!pip install python-dotenv
!pip install huggingface_hub

In [ ]:
from huggingface_hub import login

login()

In [ ]:
# # If HF token is stored in .env:
# import os
# os.environ["HF_TOKEN"] = os.getenv('HF_TOKEN')

## Load the Model and Tokenizer from Local

In [ ]:
# import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM

# # Specify the path to your mer ged model directory
# merged_model_path = "merged_model"

# # Load the tokenizer
# tokenizer = AutoTokenizer.from_pretrained(merged_model_path)

# # Load the merged model
# finetuned_model = AutoModelForCausalLM.from_pretrained(
#                  finetuned_model_id,
#                  torch_dtype=torch.float16,
#                  device_map="auto"

# model = AutoModelForCausalLM.from_pretrained(
#     merged_model_path,
#     torch_dtype=torch.float16,  # Adjust based on your saved model's dtype
#     device_map="auto"  # Automatically maps model to available devices
# )

# # If you have a GPU, ensure the model is on the GPU
# if torch.cuda.is_available():
#     model = model.cuda()

## Load the Model and Tokenizer from HF (after pushing)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

username = "jaydiaz2023"
model_id = f"{username}/gemma-2b-instruct-reasoning"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Load the model
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,  # Adjust based on your saved model's dtype
    device_map="auto"  # Automatically maps model to available devices
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Test the finetuned model

In [ ]:
# Helper functions to get the responses in proper format

def get_completion(query: str, model, tokenizer) -> str:
  device = "cuda:0"

  prompt_template = """
  <start_of_turn>user
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  {query}
  <end_of_turn>\n<start_of_turn>model


  """
  prompt = prompt_template.format(query=query)

  encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)

  model_inputs = encodeds.to(device)


  generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)
  # decoded = tokenizer.batch_decode(generated_ids)
  decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
  return decoded

In [ ]:
query = """\n\n Please answer with one of the option in the bracket. Write reasoning in between <analysis></analysis>.
           Write answer in between <answer></answer>. You are tasked with calculating the probability of a specific event occurring when rolling a die three times. Follow the steps below to solve the problem. \nWhat is the probability of getting a sum of 11 when a die is rolled three times?
           \n{'A':	'Understand the problem: We need to determine the probability of getting a sum of 11 when rolling a die three times.','B': 'Calculate total possible outcomes: A standard die has 6 faces, so each roll has 6 possible outcomes. For three rolls, the total number of outcomes is 6^3, which equals 216.',
           'C': Identify favorable outcomes: Determine all combinations of rolls that result in a sum of 11. There are 18 such combinations.', 'D': 'Calculate the probability: The probability is the ratio of favorable outcomes to the total possible outcomes. This gives us 18 / 216 = 1/12.', 'E': 'Conclusion: Therefore, the probability of getting a sum of 11 when rolling a die three times is 1/12.'}"""

result = get_completion(query=query, model=model, tokenizer=tokenizer)
print(result)

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [ ]:
print(f"{result.split('Analysis:')[-1]}")

NameError: name 'result' is not defined